# Deep-STOC: Deep Learning Model for Offshore Earthquake Prediction

This notebook presents the steps involved in tuning a Convolutional LSTM model (Deep-STOC) used for predicting the likelihood of offshore earthquakes based on remote sensing data. The model is constructed and trained using TensorFlow and Keras libraries in Python. We make use of the Keras Tuner library to perform hyperparameter tuning to optimize the model's performance.

The process in this notebook is divided into several steps:

1. **Data Loading:** The data used for training and testing the model is loaded from a specific location in the Google Drive.

2. **Data Preprocessing:** The loaded data is preprocessed, which includes reshaping and normalizing the data using MinMaxScaler from sklearn.

3. **Train-Test Split:** The preprocessed data is divided into training and testing datasets.

4. **Hypermodel Definition:** A custom HyperModel is defined using the Keras Tuner's HyperModel class. In the HyperModel, we build a ConvLSTM model with several layers where each layer's hyperparameters (like the number of filters, kernel size, activation function) are variables.

5. **Hyperparameter Tuning:** We perform hyperparameter tuning using Bayesian Optimization, searching through the hyperparameters' predefined space to find the set that results in the best validation accuracy.

6. **Model Evaluation:** The best hyperparameters and model are fetched, and the model summary is printed.

7. **Trial Details:** All the trials conducted by the tuner are displayed with their respective hyperparameters and scores.

The ultimate goal of this notebook is to find the most effective model (best hyperparameters) for predicting offshore earthquakes using remote sensing data.

Please ensure the correct versions of all required libraries are installed and that the data file paths are set correctly for successful execution.


In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras_tuner import HyperModel, RandomSearch, BayesianOptimization
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, ConvLSTM2D, Flatten

# Mount google drive for data loading
drive.mount('/content/drive')

# Load data from drive
X = np.load('drive/MyDrive/X_deepcolour7.npy', allow_pickle = True)
y = np.load('drive/MyDrive/y_deepcolour7.npy', allow_pickle = True)

# Preprocess loaded data
values = np.stack([sample[0] for sample in X])
X = values.transpose(0, 1, 2, 3, 4) 

# Display data shape
print(X.shape)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a separate MinMaxScaler for each variable (channel)
scalers = [MinMaxScaler() for _ in range(X_train.shape[-1])]

# Fit the scalers to the training data and transform the training and testing data
for i in range(X_train.shape[-1]):
    X_train[..., i] = scalers[i].fit_transform(X_train[..., i].reshape(-1, 1)).reshape(X_train[..., i].shape)
    X_test[..., i] = scalers[i].transform(X_test[..., i].reshape(-1, 1)).reshape(X_test[..., i].shape)


In [ ]:
# Define the HyperModel class for hyperparameter tuning
class MyHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        # Build the model with hyperparameters
        # hp.Int and hp.Choice are used to specify ranges for the hyperparameters
        
        model = Sequential()
        # Define first layer with variable hyperparameters
        model.add(ConvLSTM2D(
            filters=hp.Int('filters_1', 32, 128, step=32),
            kernel_size=(hp.Int('kernel_size_1_height', 2, 4), hp.Int('kernel_size_1_width', 2, 4)),
            activation=hp.Choice('activation_1', ['relu', 'tanh']),
            return_sequences=True,
            input_shape=self.input_shape
        ))
        
        # Add additional ConvLSTM2D layers based on the hyperparameter 'additional_layers'
        additional_layers = hp.Int('additional_layers', 1, 3)
        for i in range(additional_layers):
            model.add(ConvLSTM2D(
                filters=hp.Int(f'filters_{i+2}', 32, 128, step=32),
                kernel_size=(hp.Int(f'kernel_size_{i+2}_height', 2, 4), hp.Int(f'kernel_size_{i+2}_width', 2, 4)),
                activation=hp.Choice(f'activation_{i+2}', ['relu', 'tanh']),
                return_sequences=True if i < additional_layers - 1 else False
            ))

        # Flatten layer
        model.add(Flatten())
        # Dropout layer with variable rate
        model.add(Dropout(rate=hp.Float('dropout_rate', 0, 0.5, step=0.1)))
        # Final Dense layer
        model.add(Dense(1, activation='sigmoid'))
        
        # Compile the model with variable optimizer
        model.compile(
            optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model

# Initialize HyperModel with the input shape
hypermodel = MyHyperModel(input_shape=(None, X_train.shape[2], X_train.shape[3], X_train.shape[4]))

# Initialize BayesianOptimization tuner
tuner = BayesianOptimization(
    hypermodel,
    objective='val_accuracy',
    max_trials=20,
    num_initial_points=2,  # number of randomly selected hyperparameter configurations to test
    seed=43,  # to ensure reproducibility
    directory='my_dir',
    project_name='convlstm_bayesian'
)

# Execute search with training data
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)


In [ ]:
# Fetch the best hyperparameters and best model
best_hyperparameters = tuner.get_best_hyperparameters()[0]
best_model = tuner.get_best_models()[0]

# Print the best hyperparameters and the model summary
print("Best Hyperparameters: ", best_hyperparameters.values)
print(best_model.summary())

# Fetch all the trial details
all_trials = tuner.oracle.trials

# Print each trial's hyperparameters and score
for trial_id, trial in all_trials.items():
    print("Trial ", trial_id)
    print("Hyperparameters: ", trial.hyperparameters.values)
    print("Score: ", trial.score)